# Fine-tuning a Transformer Model for Multilingual Named Entity Recognition using the WikiNEuRal dataset (https://github.com/Babelscape/wikineural)

This guide will show you how to fine-tune 🤗 Transformers models for Named Entity Recognition. You will use the 🤗
Datasets library to quickly load and preprocess the WikiNEuRal dataset, getting them ready for training with PyTorch.

In [1]:
! pip install datasets transformers seqeval

# Fine-tuning a model on a token classification task

Token classification refers to the task of classifying individual tokens in a sentence. One of the most common token
classification tasks is Named Entity Recognition (NER). NER attempts to find a label for each entity in a sentence,
such as a person, location, or organization. In this notebook, you will learn how to fine-tune a model on the [WikiNEuRal](https://github.com/Babelscape/wikineural) dataset to detect new entities.

## Loading the WikiNEuRal dataset

We will use the [🤗 Datasets](https://github.com/huggingface/datasets) library to download the data and get the metric we need to use for evaluation (to compare our model to the benchmark). This can be easily done with the functions `load_dataset` and `load_metric`. 

The notebook should work with any token classification dataset provided by the 🤗 Datasets library. If you're using your own dataset defined from a JSON or csv file (see the [Datasets documentation](https://huggingface.co/docs/datasets/loading_datasets.html#from-local-files) on how to load them), it might need some adjustments in the names of the columns used.

In [2]:
from datasets import load_dataset, load_metric, concatenate_datasets

datasets = load_dataset("Babelscape/wikineural")

Using custom data configuration Babelscape--wikineural-c8d61e4987fde127
Reusing dataset parquet (/root/.cache/huggingface/datasets/parquet/Babelscape--wikineural-c8d61e4987fde127/0.0.0/1638526fd0e8d960534e2155dc54fdff8dce73851f21f031d2fb9c2cf757c121)


  0%|          | 0/27 [00:00<?, ?it/s]

The WikiNEuRal dataset has train, validation and test splits for each of the 9 languages.

To access an actual element, you need to select a split first, then give an index:

In [3]:
datasets["train_en"][0]

{'tokens': ['This',
  'division',
  'also',
  'contains',
  'the',
  'Ventana',
  'Wilderness',
  ',',
  'home',
  'to',
  'the',
  'California',
  'condor',
  '.'],
 'ner_tags': [0, 0, 0, 0, 0, 5, 6, 0, 0, 0, 0, 5, 0, 0],
 'lang': 'en'}

The labels are already coded as integer ids to be easily usable by our model, but the correspondence with the actual categories is stored the following:

In [4]:
label_list = ['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC', 'B-MISC', 'I-MISC']

labels_vocab = {'O': 0, 'B-PER': 1, 'I-PER': 2, 'B-ORG': 3, 'I-ORG': 4, 'B-LOC': 5, 'I-LOC': 6, 'B-MISC': 7, 'I-MISC': 8}
labels_vocab_reverse = {v:k for k,v in labels_vocab.items()}

At this point, in order to obtain a multilingual model, we concatenate the training, validation and test sets of all languages. 

If you want to obtain a monolingual model, you have to use a single language for each split.

In [5]:
train_dataset = concatenate_datasets([datasets["train_de"], 
                                      datasets["train_en"], 
                                      datasets["train_es"], 
                                      datasets["train_fr"],
                                      datasets["train_it"],
                                      datasets["train_nl"],
                                      datasets["train_pl"],
                                      datasets["train_pt"],
                                      datasets["train_ru"]])

val_dataset = concatenate_datasets([datasets["val_de"], 
                                    datasets["val_en"], 
                                    datasets["val_es"], 
                                    datasets["val_fr"],
                                    datasets["val_it"],
                                    datasets["val_nl"],
                                    datasets["val_pl"],
                                    datasets["val_pt"],
                                    datasets["val_ru"]])

test_dataset = concatenate_datasets([datasets["test_de"], 
                                    datasets["test_en"], 
                                    datasets["test_es"], 
                                    datasets["test_fr"],
                                    datasets["test_it"],
                                    datasets["test_nl"],
                                    datasets["test_pl"],
                                    datasets["test_pt"],
                                    datasets["test_ru"]])

We can see the training, validation and test sets all have a column for the tokens (the input texts split into words) and one column of labels for each kind of task we introduced before.

## Preprocessing the data

Before we can feed those texts to our model, we need to preprocess them. This is done by a 🤗 Transformers `Tokenizer` which will (as the name indicates) tokenize the inputs (including converting the tokens to their corresponding IDs in the pretrained vocabulary) and put it in a format the model expects, as well as generate the other inputs that model requires.

To do all of this, we instantiate our tokenizer with the `AutoTokenizer.from_pretrained` method, which will ensure:

- we get a tokenizer that corresponds to the model architecture we want to use,
- we download the vocabulary used when pretraining this specific checkpoint.

That vocabulary will be cached, so it's not downloaded again the next time we run the cell.

In [6]:
from transformers import AutoTokenizer
    
tokenizer = AutoTokenizer.from_pretrained("bert-base-multilingual-cased") 

The following assertion ensures that our tokenizer is a fast tokenizers (backed by Rust) from the 🤗 Tokenizers library. Those fast tokenizers are available for almost all models, and we will need some of the special features they have for our preprocessing.

In [7]:
import transformers
assert isinstance(tokenizer, transformers.PreTrainedTokenizerFast)

You can check which type of models have a fast tokenizer available and which don't on the [big table of models](https://huggingface.co/transformers/index.html#bigtable).

You can directly call this tokenizer on one sentence:

In [8]:
tokenizer("Hello, this is one sentence!")

{'input_ids': [101, 31178, 117, 10531, 10124, 10464, 49219, 106, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1]}

Depending on the model you selected, you will see different keys in the dictionary returned by the cell above. They don't matter much for what we're doing here (just know they are required by the model we will instantiate later), you can learn more about them in [this tutorial](https://huggingface.co/transformers/preprocessing.html) if you're interested.

If, as is the case here, your inputs have already been split into words, you should pass the list of words to your tokenzier with the argument `is_split_into_words=True`:

In [9]:
tokenizer(["Hello", ",", "this", "is", "one", "sentence", "split", "into", "words", "."], is_split_into_words=True)

{'input_ids': [101, 31178, 117, 10531, 10124, 10464, 49219, 24137, 10708, 21296, 119, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

Note that transformers are often pretrained with subword tokenizers, meaning that even if your inputs have been split into words already, each of those words could be split again by the tokenizer. Let's look at an example of that:

In [10]:
example = datasets["train_en"][0]
print(example["tokens"])

['This', 'division', 'also', 'contains', 'the', 'Ventana', 'Wilderness', ',', 'home', 'to', 'the', 'California', 'condor', '.']


In [11]:
tokenized_input = tokenizer(example["tokens"], is_split_into_words=True)
tokens = tokenizer.convert_ids_to_tokens(tokenized_input["input_ids"])
print(tokens)

['[CLS]', 'This', 'division', 'also', 'contains', 'the', 'Ve', '##ntana', 'Wilderness', ',', 'home', 'to', 'the', 'California', 'con', '##dor', '.', '[SEP]']


Here the words "Ventana" and "condor" have been split in two subtokens.

This means that we need to do some processing on our labels as the input ids returned by the tokenizer are longer than the lists of labels our dataset contain, first because some special tokens might be added (we can a `[CLS]` and a `[SEP]` above) and then because of those possible splits of words in multiple tokens:

Thankfully, the tokenizer returns outputs that have a `word_ids` method which can help us.

In [12]:
print(tokenized_input.word_ids())

[None, 0, 1, 2, 3, 4, 5, 5, 6, 7, 8, 9, 10, 11, 12, 12, 13, None]


As we can see, it returns a list with the same number of elements as our processed input ids, mapping special tokens to `None` and all other tokens to their respective word. This way, we can align the labels with the processed input ids.

In [13]:
word_ids = tokenized_input.word_ids()
aligned_labels = [-100 if i is None else example["ner_tags"][i] for i in word_ids]
print(len(aligned_labels), len(tokenized_input["input_ids"]))

18 18


Here we set the labels of all special tokens to -100 (the index that is ignored by PyTorch) and the labels of all other tokens to the label of the word they come from. Another strategy is to set the label only on the first token obtained from a given word, and give a label of -100 to the other subtokens from the same word. We propose the two strategies here, just change the value of the following flag:

In [14]:
label_all_tokens = False

We're now ready to write the function that will preprocess our samples. We feed them to the `tokenizer` with the argument `truncation=True` (to truncate texts that are bigger than the maximum size allowed by the model) and `is_split_into_words=True` (as seen above). Then we align the labels with the token ids using the strategy we picked:

In [15]:
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(examples["tokens"], truncation=True, is_split_into_words=True)

    labels = []
    for i, label in enumerate(examples["ner_tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            # Special tokens have a word id that is None. We set the label to -100 so they are automatically
            # ignored in the loss function.
            if word_idx is None:
                label_ids.append(-100)
            # We set the label for the first token of each word.
            elif word_idx != previous_word_idx:
                label_ids.append(label[word_idx])
            # For the other tokens in a word, we set the label to either the current label or -100, depending on
            # the label_all_tokens flag.
            else:
                label_ids.append(label[word_idx] if label_all_tokens else -100)
            previous_word_idx = word_idx

        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs

This function works with one or several examples. In the case of several examples, the tokenizer will return a list of lists for each key:

In [16]:
tokenize_and_align_labels(datasets['train_en'][:5])

{'input_ids': [[101, 10747, 13751, 10379, 18286, 10105, 19561, 105868, 83302, 117, 11816, 10114, 10105, 11621, 10173, 11849, 119, 102], [101, 107, 12882, 19353, 10124, 10105, 40162, 21762, 10393, 10114, 44856, 131, 10117, 14516, 117, 112, 28238, 46184, 112, 119, 102], [101, 10377, 10124, 169, 36627, 12011, 10189, 107, 72513, 169, 18458, 13275, 16612, 10189, 27024, 10108, 10105, 17956, 112, 187, 22419, 10142, 169, 103034, 10111, 22899, 11440, 10189, 34208, 10114, 10529, 81466, 117, 12400, 10169, 10105, 12723, 112, 187, 54883, 10111, 29580, 10107, 107, 119, 102], [101, 10747, 101300, 10107, 10188, 63036, 11049, 10146, 24124, 10345, 81686, 10189, 11760, 25844, 68126, 78978, 17137, 10345, 97231, 119, 102], [101, 15591, 11059, 117, 10893, 19902, 29382, 55584, 10106, 19811, 13332, 117, 13062, 24408, 10106, 19615, 13332, 117, 10111, 12275, 45982, 10106, 18372, 13332, 10529, 17087, 10407, 27279, 10798, 23590, 119, 102]], 'token_type_ids': [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

To apply this function on all the sentences (or pairs of sentences) in our dataset, we just use the `map` method of our `dataset` object we created earlier. This will apply the function on all the elements of all the splits in `dataset`, so our training, validation and testing data will be preprocessed in one single command.

In [17]:
train_tokenized = train_dataset.map(tokenize_and_align_labels, batched=True)
val_tokenized = val_dataset.map(tokenize_and_align_labels, batched=True)
test_tokenized = test_dataset.map(tokenize_and_align_labels, batched=True)

Loading cached processed dataset at /root/.cache/huggingface/datasets/parquet/Babelscape--wikineural-c8d61e4987fde127/0.0.0/1638526fd0e8d960534e2155dc54fdff8dce73851f21f031d2fb9c2cf757c121/cache-7a7f3ca2b9d388e1.arrow
Loading cached processed dataset at /root/.cache/huggingface/datasets/parquet/Babelscape--wikineural-c8d61e4987fde127/0.0.0/1638526fd0e8d960534e2155dc54fdff8dce73851f21f031d2fb9c2cf757c121/cache-e1f8fa5ad7c1c80e.arrow
Loading cached processed dataset at /root/.cache/huggingface/datasets/parquet/Babelscape--wikineural-c8d61e4987fde127/0.0.0/1638526fd0e8d960534e2155dc54fdff8dce73851f21f031d2fb9c2cf757c121/cache-81cf64fdc3a81df9.arrow


Even better, the results are automatically cached by the 🤗 Datasets library to avoid spending time on this step the next time you run your notebook. The 🤗 Datasets library is normally smart enough to detect when the function you pass to map has changed (and thus requires to not use the cache data). For instance, it will properly detect if you change the task in the first cell and rerun the notebook. 🤗 Datasets warns you when it uses cached files, you can pass `load_from_cache_file=False` in the call to `map` to not use the cached files and force the preprocessing to be applied again.

Note that we passed `batched=True` to encode the texts by batches together. This is to leverage the full benefit of the fast tokenizer we loaded earlier, which will use multi-threading to treat the texts in a batch concurrently.

## Fine-tuning the model

Now that our data is ready, we can download the pretrained model and fine-tune it. Since all our tasks are about token classification, we use the `AutoModelForTokenClassification` class. Like with the tokenizer, the `from_pretrained` method will download and cache the model for us. The only thing we have to specify is the number of labels for our problem (which we can get from the features, as seen before):

In [18]:
from transformers import AutoModelForTokenClassification, TrainingArguments, Trainer

model = AutoModelForTokenClassification.from_pretrained("bert-base-multilingual-cased", num_labels=len(label_list), label2id=labels_vocab, id2label=labels_vocab_reverse)

Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertForTokenClassification: ['cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassification were not initialized from the model checkpoint at 

The warning is telling us we are throwing away some weights (the `vocab_transform` and `vocab_layer_norm` layers) and randomly initializing some other (the `pre_classifier` and `classifier` layers). This is absolutely normal in this case, because we are removing the head used to pretrain the model on a masked language modeling objective and replacing it with a new head for which we don't have pretrained weights, so the library warns us we should fine-tune this model before using it for inference, which is exactly what we are going to do.

To instantiate a `Trainer`, we will need to define three more things. The most important is the [`TrainingArguments`](https://huggingface.co/transformers/main_classes/trainer.html#transformers.TrainingArguments), which is a class that contains all the attributes to customize the training. It requires one folder name, which will be used to save the checkpoints of the model, and all other arguments are optional:

In [19]:
model_name = "bert-base-multilingual-cased"
args = TrainingArguments(
    "wikineural-multilingual-ner",
    evaluation_strategy = "steps",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=1,
    weight_decay=0.01,
    push_to_hub=True,
    eval_steps=10000,
    save_steps=10000,
)

Here we set the evaluation to be done at the end of each epoch, tweak the learning rate,customize the batch_size and the number of epochs for training, as well as the weight decay.

Then we will need a data collator that will batch our processed examples together while applying padding to make them all the same size (each pad will be padded to the length of its longest example). There is a data collator for this task in the Transformers library, that not only pads the inputs, but also the labels:

In [20]:
from transformers import DataCollatorForTokenClassification

data_collator = DataCollatorForTokenClassification(tokenizer)

The last thing to define for our `Trainer` is how to compute the metrics from the predictions. Here we will load the [`seqeval`](https://github.com/chakki-works/seqeval) metric (which is commonly used to evaluate results on the CONLL dataset) via the Datasets library.

In [21]:
metric = load_metric("seqeval")

This metric takes list of labels for the predictions and references:

In [22]:
labels = [label_list[i] for i in example["ner_tags"]]
metric.compute(predictions=[labels], references=[labels])

{'LOC': {'precision': 1.0, 'recall': 1.0, 'f1': 1.0, 'number': 2},
 'overall_precision': 1.0,
 'overall_recall': 1.0,
 'overall_f1': 1.0,
 'overall_accuracy': 1.0}

So we will need to do a bit of post-processing on our predictions:
- select the predicted index (with the maximum logit) for each token
- convert it to its string label
- ignore everywhere we set a label of -100

The following function does all this post-processing on the result of `Trainer.evaluate` (which is a namedtuple containing predictions and labels) before applying the metric:

In [23]:
import numpy as np

def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    # Remove ignored index (special tokens)
    true_predictions = [
        [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = metric.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

Note that we drop the precision/recall/f1 computed for each category and only focus on the overall precision/recall/f1/accuracy.

Then we just need to pass all of this along with our datasets to the `Trainer`:

In [24]:
trainer = Trainer(
    model,
    args,
    train_dataset=train_tokenized,
    eval_dataset=test_tokenized,
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

/mnt/data/Wikineural/wikineural-multilingual-ner is already a clone of https://huggingface.co/sted97/wikineural-multilingual-ner. Make sure you pull the latest changes with `repo.git_pull()`.


We can now finetune our model by just calling the `train` method:

In [25]:
trainer.train()

The following columns in the training set  don't have a corresponding argument in `BertForTokenClassification.forward` and have been ignored: ner_tags, lang, tokens.
/root/miniconda3/envs/torch/lib/python3.8/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use thePyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 821600
  Num Epochs = 1
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 51350
Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently l

Step,Training Loss,Validation Loss


The `evaluate` method allows you to evaluate again on the evaluation dataset or on another dataset:

In [ ]:
trainer.evaluate()

The following columns in the evaluation set  don't have a corresponding argument in `BertForTokenClassification.forward` and have been ignored: id, pos_tags, tokens, ner_tags, chunk_tags.
***** Running Evaluation *****
  Num examples = 3454
  Batch size = 16


{'eval_loss': 0.30016785860061646,
 'eval_precision': 0.7181161911309728,
 'eval_recall': 0.7397308781869688,
 'eval_f1': 0.7287633001918715,
 'eval_accuracy': 0.9419834176806289,
 'eval_runtime': 6.5439,
 'eval_samples_per_second': 527.816,
 'eval_steps_per_second': 33.008,
 'epoch': 1.0}

To get the precision/recall/f1 computed for each category now that we have finished training, we can apply the same function as before on the result of the `predict` method:

In [ ]:
predictions, labels, _ = trainer.predict(test_tokenized)
predictions = np.argmax(predictions, axis=2)

# Remove ignored index (special tokens)
true_predictions = [
    [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
    for prediction, label in zip(predictions, labels)
]
true_labels = [
    [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
    for prediction, label in zip(predictions, labels)
]

results = metric.compute(predictions=true_predictions, references=true_labels)
results

The following columns in the test set  don't have a corresponding argument in `BertForTokenClassification.forward` and have been ignored: id, pos_tags, tokens, ner_tags, chunk_tags.
***** Running Prediction *****
  Num examples = 3454
  Batch size = 16


{'LOC': {'precision': 0.735527809307605,
  'recall': 0.7769784172661871,
  'f1': 0.7556851311953353,
  'number': 1668},
 'MISC': {'precision': 0.3394027676620539,
  'recall': 0.6638176638176638,
  'f1': 0.4491566265060241,
  'number': 702},
 'ORG': {'precision': 0.8386824324324325,
  'recall': 0.5978326309452138,
  'f1': 0.6980667838312831,
  'number': 1661},
 'PER': {'precision': 0.9492995330220146,
  'recall': 0.8800247371675943,
  'f1': 0.9133504492939666,
  'number': 1617},
 'overall_precision': 0.7181161911309728,
 'overall_recall': 0.7397308781869688,
 'overall_f1': 0.7287633001918715,
 'overall_accuracy': 0.9419834176806289}